In [3]:
import requests

api_url = 'https://development.api.wandeed.com/api/lst/search?offset=0&limit=500'
access_token='eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJydXRUeHJ5dlltOUVZcGhpRjRxak45ajFvTktLNnU4YUhuM1QySFFSUU5FIn0.eyJleHAiOjE2NjIwNDc1MDAsImlhdCI6MTY2MjAyOTUwMCwiYXV0aF90aW1lIjoxNjYyMDEzMTYwLCJqdGkiOiIyYWM0ZmVlNS1hNTgyLTQyY2MtOWYxMC1lODVhMDdhMTBlZGQiLCJpc3MiOiJodHRwczovL2RldmVsb3BtZW50LmF1dGgud2FuZGVlZC5jb20vYXV0aC9yZWFsbXMvd2FuZGVlZC1yZWFsbSIsImF1ZCI6ImFjY291bnQiLCJzdWIiOiIzMWZmODY3NC1iOGZhLTQyMmYtYWM3NC02YzFjZGI2YTUwZGUiLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJ3YW5kZWVkLWNsaWVudCIsIm5vbmNlIjoiNjgxZTBhMzktZmNlNS00OWRmLTk1NjYtMmViY2UyNjE4ZjRhIiwic2Vzc2lvbl9zdGF0ZSI6IjRjZGViZWUwLWE5MTAtNDkxMC05NDQzLTMwMWUzZGM4YjdjMyIsImFjciI6IjAiLCJhbGxvd2VkLW9yaWdpbnMiOlsiaHR0cHM6Ly8qLndhbmRlZWQuY29tLyoiLCJodHRwczovLyouYXBpLndhbmRlZWQuY29tLyoiLCIqIiwiaHR0cHM6Ly8qLmFkbWluLndhbmRlZWQuY29tLyoiLCJodHRwczovLyouYXV0aC53YW5kZWVkLmNvbSJdLCJyZWFsbV9hY2Nlc3MiOnsicm9sZXMiOlsiZGVmYXVsdC1yb2xlcy13YW5kZWVkLXJlYWxtIiwib2ZmbGluZV9hY2Nlc3MiLCJ1bWFfYXV0aG9yaXphdGlvbiJdfSwicmVzb3VyY2VfYWNjZXNzIjp7ImFjY291bnQiOnsicm9sZXMiOlsibWFuYWdlLWFjY291bnQiLCJtYW5hZ2UtYWNjb3VudC1saW5rcyIsInZpZXctcHJvZmlsZSJdfX0sInNjb3BlIjoib3BlbmlkIGVtYWlsIHByb2ZpbGUiLCJzaWQiOiI0Y2RlYmVlMC1hOTEwLTQ5MTAtOTQ0My0zMDFlM2RjOGI3YzMiLCJ1dGxfc3BrdXRpbGlzYXRldXIiOjEsInV0bF91dGlsaXNhdGV1cl9yb2xlcyI6IlsxXSIsInV0bF9jcHJlbm9tIjoiR2F5bG9yZCIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJ1dGxfc2FwcGFydGVuYW5jZSI6WzIsMzhdLCJ1dGxfY25vbSI6IlBldGl0IiwicHJlZmVycmVkX3VzZXJuYW1lIjoiZ2F5bG9yZC5wZXRpdEB0ZWFtYmVyLmZyIiwibG9jYWxlIjoiZnIiLCJnaXZlbl9uYW1lIjoiR2F5bG9yZCIsInV0bF9zZmtpbnN0YW5jZSI6MSwidXRsX3NhZG1pbmlzdHJlciI6W10sIm5hbWUiOiJHYXlsb3JkIFBldGl0IiwidXRsX3Nkcm9pdHNhY2NlcyI6WzYwLDMsNCw1LDE1LDI1LDI2LDMzLDM0LDM1LDM2LDM3LDQwLDQxLDQ0LDQ1LDQ2LDQ3XSwiZmFtaWx5X25hbWUiOiJQZXRpdCIsImVtYWlsIjoiZ2F5bG9yZC5wZXRpdEB0ZWFtYmVyLmZyIiwidXNlcl9ncm91cHMiOlsyLDM4LDYwLDMsNCw1LDE1LDI1LDI2LDMzLDM0LDM1LDM2LDM3LDQwLDQxLDQ0LDQ1LDQ2LDQ3XX0.rzhDdHuidbki30T98ktirs7hpuRAGfx5s5QJdoo44sIKkBBfZaywYGwt_nNiCom3xNpFbI3eChye7HjbVdz6FqegJhP3jVSJAtu7xpCLJd0CebzXqZdg9eYMfiehXv7an_jyAHS7VcrXj5beOzAghdruxpUvSFzYXER_-nQfw3QSs0--DceFGFdOZLJPkqOw0aXMSG5L8WqdSSJm75LroJi3K59kxRkJkSOgWngRYWOKG9Q0UXoqvSwHrAT4oTjGGrbhMJMYOUIV4-mPUyEQ0oc4mtqzRcqUAd_yFaGPGVGRfWY8bbVyWBD9YbwmeOf-NymeY-eiIMSjaViDWu97uw'

data = {
    "select": [
        "utl_spkutilisateur",
        "int_sfkprojet"
    ],
    "from": "lst_vevenement_PY",
    "where": {
        "condition": "and",
        "rules": [
            {
                "label": "evt_xdate_debut",
                "field": "evt_xdate_debut",
                "operator": "morethan",
                "type": "boolean",
                "value": "01/12/2022"
            },
            {
                "label": "evt_xdate_fin",
                "field": "evt_xdate_fin",
                "operator": "lessthan",
                "type": "number",
                "value": "30/12/2022"
            }
        ]
    }
}


In [250]:
data = {'select':['emc_sfkutilisateur','emc_sfkarticle','emc_sniveau'],
        'from': 'lst_vcompetence_PY',
}
resp = requests.post(api_url, headers=headers, json = data)
print(resp)

data = {
    "select": [
        "utl_spkutilisateur",
        "int_sfkprojet"
    ],
    "from": "lst_vevenement_PY",
    "where": {
        "condition": "and",
        "rules": [
            {
                "label": "evt_xdate_debut",
                "field": "evt_xdate_debut",
                "operator": "morethan",
                "type": "boolean",
                "value": "01/12/2022"
            },
            {
                "label": "evt_xdate_fin",
                "field": "evt_xdate_fin",
                "operator": "lessthan",
                "type": "number",
                "value": "30/12/2022"
            }
        ]
    }
}
resp = requests.post(api_url, headers=headers, json = data)
print(resp)

<Response [412]>
<Response [412]>


## en-tête des requêtes :

In [241]:
api_url = 'https://development.api.wandeed.com/api/lst/search?offset=0&limit=500'
access_token='eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJydXRUeHJ5dlltOUVZcGhpRjRxak45ajFvTktLNnU4YUhuM1QySFFSUU5FIn0.eyJleHAiOjE2NjIwNDc1MDAsImlhdCI6MTY2MjAyOTUwMCwiYXV0aF90aW1lIjoxNjYyMDEzMTYwLCJqdGkiOiIyYWM0ZmVlNS1hNTgyLTQyY2MtOWYxMC1lODVhMDdhMTBlZGQiLCJpc3MiOiJodHRwczovL2RldmVsb3BtZW50LmF1dGgud2FuZGVlZC5jb20vYXV0aC9yZWFsbXMvd2FuZGVlZC1yZWFsbSIsImF1ZCI6ImFjY291bnQiLCJzdWIiOiIzMWZmODY3NC1iOGZhLTQyMmYtYWM3NC02YzFjZGI2YTUwZGUiLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJ3YW5kZWVkLWNsaWVudCIsIm5vbmNlIjoiNjgxZTBhMzktZmNlNS00OWRmLTk1NjYtMmViY2UyNjE4ZjRhIiwic2Vzc2lvbl9zdGF0ZSI6IjRjZGViZWUwLWE5MTAtNDkxMC05NDQzLTMwMWUzZGM4YjdjMyIsImFjciI6IjAiLCJhbGxvd2VkLW9yaWdpbnMiOlsiaHR0cHM6Ly8qLndhbmRlZWQuY29tLyoiLCJodHRwczovLyouYXBpLndhbmRlZWQuY29tLyoiLCIqIiwiaHR0cHM6Ly8qLmFkbWluLndhbmRlZWQuY29tLyoiLCJodHRwczovLyouYXV0aC53YW5kZWVkLmNvbSJdLCJyZWFsbV9hY2Nlc3MiOnsicm9sZXMiOlsiZGVmYXVsdC1yb2xlcy13YW5kZWVkLXJlYWxtIiwib2ZmbGluZV9hY2Nlc3MiLCJ1bWFfYXV0aG9yaXphdGlvbiJdfSwicmVzb3VyY2VfYWNjZXNzIjp7ImFjY291bnQiOnsicm9sZXMiOlsibWFuYWdlLWFjY291bnQiLCJtYW5hZ2UtYWNjb3VudC1saW5rcyIsInZpZXctcHJvZmlsZSJdfX0sInNjb3BlIjoib3BlbmlkIGVtYWlsIHByb2ZpbGUiLCJzaWQiOiI0Y2RlYmVlMC1hOTEwLTQ5MTAtOTQ0My0zMDFlM2RjOGI3YzMiLCJ1dGxfc3BrdXRpbGlzYXRldXIiOjEsInV0bF91dGlsaXNhdGV1cl9yb2xlcyI6IlsxXSIsInV0bF9jcHJlbm9tIjoiR2F5bG9yZCIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJ1dGxfc2FwcGFydGVuYW5jZSI6WzIsMzhdLCJ1dGxfY25vbSI6IlBldGl0IiwicHJlZmVycmVkX3VzZXJuYW1lIjoiZ2F5bG9yZC5wZXRpdEB0ZWFtYmVyLmZyIiwibG9jYWxlIjoiZnIiLCJnaXZlbl9uYW1lIjoiR2F5bG9yZCIsInV0bF9zZmtpbnN0YW5jZSI6MSwidXRsX3NhZG1pbmlzdHJlciI6W10sIm5hbWUiOiJHYXlsb3JkIFBldGl0IiwidXRsX3Nkcm9pdHNhY2NlcyI6WzYwLDMsNCw1LDE1LDI1LDI2LDMzLDM0LDM1LDM2LDM3LDQwLDQxLDQ0LDQ1LDQ2LDQ3XSwiZmFtaWx5X25hbWUiOiJQZXRpdCIsImVtYWlsIjoiZ2F5bG9yZC5wZXRpdEB0ZWFtYmVyLmZyIiwidXNlcl9ncm91cHMiOlsyLDM4LDYwLDMsNCw1LDE1LDI1LDI2LDMzLDM0LDM1LDM2LDM3LDQwLDQxLDQ0LDQ1LDQ2LDQ3XX0.rzhDdHuidbki30T98ktirs7hpuRAGfx5s5QJdoo44sIKkBBfZaywYGwt_nNiCom3xNpFbI3eChye7HjbVdz6FqegJhP3jVSJAtu7xpCLJd0CebzXqZdg9eYMfiehXv7an_jyAHS7VcrXj5beOzAghdruxpUvSFzYXER_-nQfw3QSs0--DceFGFdOZLJPkqOw0aXMSG5L8WqdSSJm75LroJi3K59kxRkJkSOgWngRYWOKG9Q0UXoqvSwHrAT4oTjGGrbhMJMYOUIV4-mPUyEQ0oc4mtqzRcqUAd_yFaGPGVGRfWY8bbVyWBD9YbwmeOf-NymeY-eiIMSjaViDWu97uw'

headers={'Authorization': f'Bearer {access_token}',
             'Content-Type':'application/json'}

## requete pour obtenir les compétences utilisateurs :

In [248]:
data = {'select':['emc_sfkutilisateur','emc_sfkarticle','emc_sniveau'],
        'from': 'lst_vcompetence_PY'
}
resp = requests.post(api_url, headers=headers, json = data)
print(resp)


data = {'select':['utl_spkutilisateur','utl_sdispo'],
        'from': 'lst_vdispo_py'}
resp = requests.post(api_url, headers=headers, json = data)
print(resp)

<Response [412]>
<Response [200]>


In [231]:
import pandas as pd
def get_competences_utilisateurs(): # later : param list_utilisateur
    data = {'select':['emc_sfkutilisateur','emc_sfkarticle','emc_sniveau'],'from': 'lst_vcompetence_PY'}
    print('envoi de la requete pour les dispos utilisateurs')
    resp = requests.post(api_url, headers=headers, json = data)
    if resp.status_code != 200 :
        print('Erreur récupération des données sur le serveur')
        
    else: 
        print('Requête effectuée aves succès.')
        res = resp.json()['result']
        df = pd.DataFrame(res)
        return df

get_dispo_utilisateur()    

envoi de la requete pour les dispos utilisateurs
Erreur récupération des données sur le serveur


## requete pour obtenir les dispos utilisateurs :

In [217]:
import pandas as pd
def get_dispo_utilisateur(): # later : param list_utilisateur
    data = {'select':['utl_spkutilisateur','utl_sdispo'],'from': 'lst_vdispo_py'}
    print('envoi de la requete pour les dispos utilisateurs')
    resp = requests.post(api_url, headers=headers, json = data)
    if resp.status_code != 200 :
        print('Erreur récupération des données sur le serveur')
        
    else: 
        print('Requête effectuée aves succès.')
        res = resp.json()['result']
        df = pd.DataFrame(res)
        return df

get_dispo_utilisateur()    

envoi de la requete pour les dispos utilisateurs
Requête effectuée aves succès.


,utl_spkutilisateur,utl_sdispo
0,4,13
1,6,6
2,8,17
3,9,98
4,11,3
5,5,70
6,24,78
7,10,34
8,7,19
9,1,58


### requete pour obtenir la matrice projet

In [230]:
import pandas as pd
def get_matrice_projet(): # later : param list_utilisateur
    data = {
    "select": [
        "utl_spkutilisateur",
        "int_sfkprojet"],
    "from": "lst_vprojet_utilisateur_py",}
    
    print('envoi de la requete pour la matrice projet')
    resp = requests.post(api_url, headers=headers, json = data)
    if resp.status_code != 200 :
        print('Erreur récupération des données sur le serveur')
        
    else: 
        print('Requête effectuée aves succès.')
        res = resp.json()['result']
        df = pd.DataFrame(res)
        return df

get_matrice_projet()    

envoi de la requete pour la matrice projet
Requête effectuée aves succès.


,utl_spkutilisateur,int_sfkprojet
0,1,10090
1,4,10090
2,3,10090
3,2,10090
4,2,10095
...,...,...
61,9,10120
62,1,10232
63,1,10229
64,1,10228
